In [1]:
#   ___                  _                
#  / _/______ ____  ____(_)__ _______     
# / _/ __/ _ `/ _ \/ __/ (_-</ __/ _ \    
#/_//_/  \_,_/_//_/\__/_/___/\__/\___/    
# ___ _____(_)__ ___ ____  / /_(_)       
# / _ `/ __/ (_-</ _ `/ _ \/ __/ /        
# \_, /_/ /_/___/\_,_/_//_/\__/_/         
#/___/
#
#Samee Lab @ Baylor College Of Medicine
#francisco.grisanticanozo@bcm.edu

In [4]:
library(Seurat)
library(dplyr)
library(Matrix)
library(SeuratDisk)
library(methods)
library(ggplot2)
library(preprocessCore)

options(stringsAsFactors = FALSE)
future::plan("multiprocess", workers = 20) # do parallel

In [5]:
#Arguments

file_path_train_data <- '../Data/Single_Cell_Reference/adata_processed_for_rds.h5ad'
file_path_predict_data <- '../Data/Merfish/merfish_for_rds.h5ad'


file_out_path_train_data <- '../Data/Single_Cell_Reference/adata_processed_qn.h5ad'
file_out_path_predict_data <- '../Data/Merfish/merfish_qn.h5ad'


#Pipeline

Convert(
  file_path_train_data,
  dest = "h5seurat",
  assay = "RNA",
  overwrite = TRUE,
  verbose = TRUE
)

Convert(
  file_path_predict_data,
  dest = "h5seurat",
  assay = "RNA",
  overwrite = TRUE,
  verbose = TRUE
)

object_r <- LoadH5Seurat(gsub("h5ad", "h5seurat", file_path_train_data))
object_r_m <- t(as.matrix(GetAssayData(object_r, assay= 'RNA', slot = "scale.data")))
object_t <- LoadH5Seurat(gsub("h5ad", "h5seurat", file_path_predict_data))
object_t_m <- t(as.matrix(GetAssayData(object_t, assay= 'RNA', slot = "scale.data")))


quantileNormalize <- function(
  seqFISH_expression,
  scRNAseq_expression,
  normalize_scRNAseq = TRUE
){

  seqFISH_genes <- colnames(seqFISH_expression)
  seqFISH_cells <- rownames(seqFISH_expression)
  scRNAseq_genes <- colnames(scRNAseq_expression)
  scRNAseq_cells <- rownames(scRNAseq_expression)

  if(normalize_scRNAseq) scRNAseq_expression <- normalize.quantiles(scRNAseq_expression)

  scRNAseq_target <- normalize.quantiles.determine.target(t(scRNAseq_expression))
  seqFISH_expression <- normalize.quantiles.use.target(seqFISH_expression, scRNAseq_target)

  seqFISH_expression <- round(seqFISH_expression, 4)
  scRNAseq_expression <- round(scRNAseq_expression, 4)

  colnames(seqFISH_expression) <- seqFISH_genes
  rownames(seqFISH_expression) <- seqFISH_cells
  colnames(scRNAseq_expression) <- scRNAseq_genes
  rownames(scRNAseq_expression) <- scRNAseq_cells

  result <- list(seqFISH_expression = seqFISH_expression, scRNAseq_expression = scRNAseq_expression)

  return(result)

}

#qn_data <- quantileNormalize(
#  seqFISH_expression = methods::as(object_t, 'matrix'),
#  scRNAseq_expression = methods::as(object_r, 'matrix'),
#)

qn_data <- quantileNormalize(
  seqFISH_expression = object_t_m,
  scRNAseq_expression = object_r_m
)

seqFISH_expression <- qn_data$seqFISH_expression
scRNAseq_expression <- qn_data$scRNAseq_expression

object <- CreateSeuratObject(t(seqFISH_expression))
SaveH5Seurat(object, 
             filename = gsub("h5ad", "h5seurat", file_out_path_predict_data),
             overwrite = TRUE)
Convert(gsub("h5ad", "h5seurat",file_out_path_predict_data), 
        dest = "h5ad",
        overwrite = TRUE)

object <- CreateSeuratObject(t(scRNAseq_expression))
SaveH5Seurat(object, 
             filename = gsub("h5ad", "h5seurat", file_out_path_train_data),
             overwrite = TRUE)
Convert(gsub("h5ad", "h5seurat", file_out_path_train_data), 
        dest = "h5ad",
        overwrite = TRUE)

Warning message:
“Unknown file type: h5ad”
Creating h5Seurat file for version 3.1.5.9900

Adding X as scale.data

Adding raw/X as data

Adding raw/X as counts

Adding meta.features from raw/var

Warning message:
“Unknown file type: h5ad”
Creating h5Seurat file for version 3.1.5.9900

Adding X as scale.data

Adding raw/X as data

Adding raw/X as counts

Adding meta.features from raw/var

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results

Warning message:
“Overwriting previous file ../Data/Merfish/merfish_qn.h5seurat”
Cre